In [1]:
import pandas as pd
import numpy as np
import sys, os
from tqdm.notebook import tqdm
from datetime import timedelta
import matplotlib.pyplot as plt
import pickle

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

In [2]:
d_items         = pd.read_parquet(icu+'d_items.parquet')
inputevents     = pd.read_parquet(icu+'inputevents.parquet')

In [3]:
d_antibiotics = d_items[(d_items['linksto']=='inputevents')&(d_items['category']=='Antibiotics')].itemid.unique()

In [7]:
inputevents['starttime'] = pd.to_datetime(inputevents['starttime'])

In [8]:
inputevents[inputevents['itemid']==d_antibiotics[0]][['subject_id','hadm_id','stay_id','starttime','itemid']]

,subject_id,hadm_id,stay_id,starttime,itemid
8,10000980,26913865,39765666,2189-06-27 07:40:00,225798
24,10001217,24597018,37067082,2157-11-21 08:03:00,225798
36,10001217,24597018,37067082,2157-11-21 17:29:00,225798
40,10001217,24597018,37067082,2157-11-20 20:19:00,225798
48,10001217,27703517,34592300,2157-12-19 20:00:00,225798
...,...,...,...,...,...
8978670,19999840,21033226,38978960,2164-09-15 20:00:00,225798
8978707,19999840,21033226,38978960,2164-09-15 08:00:00,225798
8978743,19999840,21033226,38978960,2164-09-16 08:32:00,225798
8978784,19999840,21033226,38978960,2164-09-17 08:07:00,225798


In [33]:
d_items[d_items['itemid']==i].label.unique()

array(['Vancomycin'], dtype=object)

In [46]:
def ABduration(d_antibiotics,d_items,inputevents):
    for i in tqdm(d_antibiotics):
        tmp = inputevents[inputevents['itemid']==i][['stay_id','starttime','itemid']]
        tmp.sort_values(by=['stay_id','starttime'],inplace=True)
        tmp.reset_index(inplace=True,drop=True)
        tmp = pd.concat([tmp.shift(1),tmp],axis=1)
        tmp.columns = ['stay_id_x','starttime_x','itemid_x','stay_id_y','starttime_y','itemid_y']        
        
        tmp = tmp[tmp['stay_id_x'] == tmp['stay_id_y']]
        tmp['timediff'] = tmp['starttime_y'] - tmp['starttime_x']
        tmp['timediff_days'] = tmp.timediff.dt.total_seconds()/(60*60*24)

        print(d_items[d_items['itemid']==i].label.unique()[0],':',"%.2f"%(tmp.timediff_days.mean()*24),"hours")

ABduration(d_antibiotics,d_items,inputevents)

  0%|          | 0/55 [00:00<?, ?it/s]

Vancomycin : 17.54 hours
Acyclovir : 12.08 hours
Ambisome : 28.68 hours
Amikacin : 39.89 hours
Ampicillin : 6.53 hours
Ampicillin/Sulbactam (Unasyn) : 7.59 hours
Atovaquone : 40.59 hours
Azithromycin : 26.63 hours
Aztreonam : 10.30 hours
Caspofungin : 26.68 hours
Cefazolin : 10.64 hours
Cefepime : 15.18 hours
Ceftazidime : 13.96 hours
Ceftriaxone : 25.43 hours
Chloroquine : nan hours
Ciprofloxacin : 17.22 hours
Clindamycin : 8.73 hours
Colistin : 17.06 hours
Daptomycin : 34.84 hours
Doxycycline : 13.49 hours
Erythromycin : 10.35 hours
Ethambutol : 148.03 hours
Fluconazole : 30.09 hours
Foscarnet : 30.99 hours
Gancyclovir : 21.35 hours
Gentamicin : 15.25 hours
Imipenem/Cilastatin : 8.77 hours
Isoniazid : 180.44 hours
Levofloxacin : 31.90 hours
Linezolid : 14.83 hours
Mefloquine : nan hours
Meropenem : 10.49 hours
Metronidazole : 10.11 hours
Micafungin : 31.26 hours
Moxifloxacin : 79.38 hours
Nafcillin : 5.00 hours
Oxacillin : nan hours
Penicillin G potassium : 4.43 hours
Piperacillin : 

In [12]:
d_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,None,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,None,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4009,230077,Documented on DC Summary (PICC),Documented on DC Summary (PICC),chartevents,PICC Line Insertion,None,Text,NaN,NaN
4010,230078,Documented on DC Summary (Thora),Documented on DC Summary (Thora),chartevents,Thoracentesis,None,Text,NaN,NaN
4011,230083,Replacement Fluid (Pre-Filter),Replacement Fluid (Pre-Filter),chartevents,Dialysis,None,Text,NaN,NaN
4012,230084,Replacement Fluid (Post-Filter),Replacement Fluid (Post-Filter),chartevents,Dialysis,None,Text,NaN,NaN


In [48]:
len(inputevents[inputevents['itemid'].isin(d_antibiotics)].stay_id.unique())

44636